In [1]:
# changing core directory
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
os.chdir('..')

In [2]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
import datetime

%load_ext autoreload
%autoreload

# beauty

In [3]:
path = 'checkpoints/beauty'
all_files = [file for file in os.listdir(path)  if ".csv" in file ]
all_files

['item_not_error|model=sasrec_seed=0_final=items|agg=mean__head=adding__freeze=True.csv',
 'item_not_error|model=sasrec_seed=0_final=items.csv',
 'item_not_error|model=sasrec_seed=0_final=items|agg=mean__head=adding__freeze=False.csv',
 'item_not_error|model=sasrec_seed=0_final=items|agg=learnable_attention__head=adding__freeze=False.csv',
 'item_not_error|model=sasrec_seed=0_final=items|agg=attention__head=adding__freeze=False.csv',
 'item_not_error|model=sasrec_seed=0_final=items|agg=max__head=adding__freeze=False.csv',
 'item_not_error|model=sasrec_seed=0_final=items|agg=max__head=adding__freeze=True.csv',
 'item_not_error|model=sasrec_seed=0_final=items|agg=kernel_attention__head=adding__freeze=True.csv',
 'item_not_error|model=sasrec_seed=0_final=items|agg=kernel_attention__head=adding__freeze=False.csv',
 'item_not_error|model=sasrec_seed=0_final=items|agg=learnable_attention__head=adding__freeze=True.csv',
 'item_not_error|model=sasrec_seed=0_final=items|agg=attention__head=addi

In [4]:
all_res = pd.DataFrame([])

for file in all_files:
    df = pd.read_csv(os.path.join(path, file))
    df['dataset'] = 'movielens_1m'
    df['file_name'] = file
    if len(file.split('|')) > 1:
        hp = file.split('|')[1]
        method = hp
        method = method.replace('.csv', '').replace('agg=', '')
        df['head'] = 'replace'
        df['agg_method'] = method
    else:
        df['head'] = '-'
        df['agg_method'] = 'no_agg'

    all_res = pd.concat([all_res, df])

all_res_agg = all_res

exp_mask = all_res_agg['file_name'].str.split('|').apply(lambda x: len(x)) == 3
print(len(all_res_agg))
all_res_agg = all_res_agg[exp_mask]
print(len(all_res_agg))


split_values = all_res_agg['file_name'].str.split('|', n=2, expand=True)
all_res_agg[['exp_name', 'base_model', 'exp_hp']] = split_values


all_res_agg = all_res_agg[all_res_agg['exp_name'] == 'item_not_error']

all_res_agg['head_method'] =  all_res_agg['exp_hp'].str.split('__', n=2, expand=True)[1] \
    .str.split('__', expand=True)[0] \
    .str.replace('head=', '')

all_res_agg['agg_method'] =  all_res_agg['exp_hp'].str.split('__', n=2, expand=True)[0] \
    .str.split('__', expand=True)[0] \
    .str.replace('agg=', '')

all_res_agg['freeze'] =  all_res_agg['exp_hp'].str.split('__', n=2, expand=True)[2] \
    .str.split('__', expand=True)[0] \
    .str.replace('freeze=', '') \
    .str.replace('.csv', '') \


baselines = [f'item_not_error|model=sasrec_seed={seed}_final=items.csv' for seed in [0, 1, 2]]
all_res_baseline = all_res[all_res['file_name'].isin(baselines)]
all_res_baseline[['exp_name', 'exp_hp', 'head_method', 'agg_method', 'freeze']] = \
'item_search', '', 'no_head', 'no_agg', 'False'

all_res_baseline['base_model'] = all_res_baseline['file_name'].str.split('|', expand=True)[1].str.replace('.csv', '')

all_res_agg = pd.concat([all_res_agg, all_res_baseline])

all_res_agg['seed'] = all_res_agg['base_model'].str.split('_', expand=True)[1].str.replace('seed=', '').astype(int)

all_res_agg

51
30


/tmp/ipykernel_1727/2677695248.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_res_agg[['exp_name', 'base_model', 'exp_hp']] = split_values
/tmp/ipykernel_1727/2677695248.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_res_agg[['exp_name', 'base_model', 'exp_hp']] = split_values
/tmp/ipykernel_1727/2677695248.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

,Mode,Epoch,Train_Loss,Val_Loss,Val_NDCG,dataset,file_name,head,agg_method,exp_name,base_model,exp_hp,head_method,freeze,seed
0,external,0,100000.0000,10.1244,0.021783,movielens_1m,item_not_error|model=sasrec_seed=0_final=items...,replace,mean,item_not_error,model=sasrec_seed=0_final=items,agg=mean__head=adding__freeze=True.csv,adding,True,0
0,external,2,9.7994,10.0870,0.022239,movielens_1m,item_not_error|model=sasrec_seed=0_final=items...,replace,mean,item_not_error,model=sasrec_seed=0_final=items,agg=mean__head=adding__freeze=False.csv,adding,False,0
1,external,5,9.6879,10.1975,0.022120,movielens_1m,item_not_error|model=sasrec_seed=0_final=items...,replace,mean,item_not_error,model=sasrec_seed=0_final=items,agg=mean__head=adding__freeze=False.csv,adding,False,0
2,external,8,9.6583,10.2194,0.022560,movielens_1m,item_not_error|model=sasrec_seed=0_final=items...,replace,mean,item_not_error,model=sasrec_seed=0_final=items,agg=mean__head=adding__freeze=False.csv,adding,False,0
0,external,2,9.8011,10.0818,0.022659,movielens_1m,item_not_error|model=sasrec_seed=0_final=items...,replace,learnable_attention,item_not_error,model=sasrec_seed=0_final=items,agg=learnable_attention__head=adding__freeze=F...,adding,False,0
1,external,5,9.6897,10.1288,0.023179,movielens_1m,item_not_error|model=sasrec_seed=0_final=items...,replace,learnable_attention,item_not_error,model=sasrec_seed=0_final=items,agg=learnable_attention__head=adding__freeze=F...,adding,False,0
2,external,8,9.6588,10.1618,0.022537,movielens_1m,item_not_error|model=sasrec_seed=0_final=items...,replace,learnable_attention,item_not_error,model=sasrec_seed=0_final=items,agg=learnable_attention__head=adding__freeze=F...,adding,False,0
0,external,2,9.7996,10.0857,0.022261,movielens_1m,item_not_error|model=sasrec_seed=0_final=items...,replace,attention,item_not_error,model=sasrec_seed=0_final=items,agg=attention__head=adding__freeze=False.csv,adding,False,0
1,external,5,9.6876,10.1912,0.022113,movielens_1m,item_not_error|model=sasrec_seed=0_final=items...,replace,attention,item_not_error,model=sasrec_seed=0_final=items,agg=attention__head=adding__freeze=False.csv,adding,False,0
2,external,8,9.6580,10.2073,0.022525,movielens_1m,item_not_error|model=sasrec_seed=0_final=items...,replace,attention,item_not_error,model=sasrec_seed=0_final=items,agg=attention__head=adding__freeze=False.csv,adding,False,0


In [5]:
# seed filter

all_res_agg = all_res_agg[all_res_agg['seed'] == 0]

In [6]:
agg_gr_metrics = all_res_agg.groupby(['freeze', "head_method", "agg_method"], as_index=False) \
    .agg({'Val_NDCG': 'max'}) \
    .sort_values('Val_NDCG') \
    # .groupby(['freeze', "head_method", "agg_method"], as_index=False) \
    # .agg({'Val_NDCG': ['mean', 'std']}) \
    # .sort_values('Val_NDCG')
agg_gr_metrics['freeze'] = agg_gr_metrics['freeze'] == 'True'

agg_max_gr_metrics = agg_gr_metrics.groupby(['freeze', "head_method", "agg_method"], as_index=False).agg({'Val_NDCG': ['mean', 'std']})

agg_max_gr_metrics

freeze head_method             agg_method  Val_NDCG    
                                                  mean std
0   False      adding              attention  0.022525 NaN
1   False      adding       kernel_attention  0.022689 NaN
2   False      adding    learnable_attention  0.023179 NaN
3   False      adding                    max  0.022672 NaN
4   False      adding                   mean  0.022560 NaN
5   False      adding  symmetrical_attention  0.023066 NaN
6   False     no_head                 no_agg  0.021800 NaN
7    True      adding              attention  0.021822 NaN
8    True      adding       kernel_attention  0.021814 NaN
9    True      adding    learnable_attention  0.021845 NaN
10   True      adding                    max  0.021799 NaN
11   True      adding                   mean  0.021783 NaN
12   True      adding  symmetrical_attention  0.021846 NaN

In [7]:
agg_gr_metrics_fr = agg_max_gr_metrics[(agg_max_gr_metrics['freeze']) | (agg_max_gr_metrics['agg_method'] == 'no_agg')]
agg_gr_metrics_not_fr = agg_max_gr_metrics[~agg_max_gr_metrics['freeze']]


agg_gr_metrics_not_fr.sort_values([('Val_NDCG', 'mean')])

freeze head_method             agg_method  Val_NDCG    
                                                 mean std
6  False     no_head                 no_agg  0.021800 NaN
0  False      adding              attention  0.022525 NaN
4  False      adding                   mean  0.022560 NaN
3  False      adding                    max  0.022672 NaN
1  False      adding       kernel_attention  0.022689 NaN
5  False      adding  symmetrical_attention  0.023066 NaN
2  False      adding    learnable_attention  0.023179 NaN

In [8]:
agg_gr_metrics_fr.sort_values([('Val_NDCG', 'mean')])

freeze head_method             agg_method  Val_NDCG    
                                                  mean std
11   True      adding                   mean  0.021783 NaN
10   True      adding                    max  0.021799 NaN
6   False     no_head                 no_agg  0.021800 NaN
8    True      adding       kernel_attention  0.021814 NaN
7    True      adding              attention  0.021822 NaN
9    True      adding    learnable_attention  0.021845 NaN
12   True      adding  symmetrical_attention  0.021846 NaN

# not error

In [3]:
path = 'checkpoints/movielens_1m'
all_files = [file for file in os.listdir(path)  if ".csv" in file ]
all_files

['item_add_alpha|model=sasrec_seed=0_final=items|agg=attention__head=over__freeze=True.csv',
 'item_not_error|model=sasrec_seed=0_final=items|agg=exp_learnable_hawkes__head=adding__freeze=True.csv',
 'item_add_alpha|model=sasrec_seed=2_final=items|agg=mean__head=over__freeze=True.csv',
 'item_not_error|model=sasrec_seed=0_final=items|agg=max__head=over__freeze=True.csv',
 'item_not_error|model=sasrec_seed=0_final=items|agg=kernel_attention__head=over__freeze=False.csv',
 'item_add_alpha|model=sasrec_seed=1_final=items|agg=learnable_attention__head=over__freeze=True.csv',
 'item_not_error|model=sasrec_seed=1_final=items|agg=mean__head=adding__freeze=True.csv',
 'item_add_alpha|model=sasrec_seed=0_final=items|agg=mean__head=adding__freeze=True.csv',
 'item_add_alpha|model=sasrec_seed=1_final=items|agg=symmetrical_attention__head=adding__freeze=False.csv',
 'item_not_error|model=sasrec_seed=0_final=items|agg=mean__head=adding__freeze=True.csv',
 'item_add_alpha|model=sasrec_seed=1_final=i

In [4]:
all_res = pd.DataFrame([])

for file in all_files:
    df = pd.read_csv(os.path.join(path, file))
    df['dataset'] = 'movielens_1m'
    df['file_name'] = file
    if len(file.split('|')) > 1:
        hp = file.split('|')[1]
        method = hp
        method = method.replace('.csv', '').replace('agg=', '')
        df['head'] = 'replace'
        df['agg_method'] = method
    else:
        df['head'] = '-'
        df['agg_method'] = 'no_agg'

    all_res = pd.concat([all_res, df])

all_res_agg = all_res

exp_mask = all_res_agg['file_name'].str.split('|').apply(lambda x: len(x)) == 3
print(len(all_res_agg))
all_res_agg = all_res_agg[exp_mask]
print(len(all_res_agg))


split_values = all_res_agg['file_name'].str.split('|', n=2, expand=True)
all_res_agg[['exp_name', 'base_model', 'exp_hp']] = split_values


all_res_agg = all_res_agg[all_res_agg['exp_name'] == 'item_not_error']

all_res_agg['head_method'] =  all_res_agg['exp_hp'].str.split('__', n=2, expand=True)[1] \
    .str.split('__', expand=True)[0] \
    .str.replace('head=', '')

all_res_agg['agg_method'] =  all_res_agg['exp_hp'].str.split('__', n=2, expand=True)[0] \
    .str.split('__', expand=True)[0] \
    .str.replace('agg=', '')

all_res_agg['freeze'] =  all_res_agg['exp_hp'].str.split('__', n=2, expand=True)[2] \
    .str.split('__', expand=True)[0] \
    .str.replace('freeze=', '') \
    .str.replace('.csv', '') \


baselines = [f'item_not_error|model=sasrec_seed={seed}_final=items.csv' for seed in [0, 1, 2]]
all_res_baseline = all_res[all_res['file_name'].isin(baselines)]
all_res_baseline[['exp_name', 'exp_hp', 'head_method', 'agg_method', 'freeze']] = \
'item_search', '', 'no_head', 'no_agg', 'False'

all_res_baseline['base_model'] = all_res_baseline['file_name'].str.split('|', expand=True)[1].str.replace('.csv', '')

all_res_agg = pd.concat([all_res_agg, all_res_baseline])

all_res_agg['seed'] = all_res_agg['base_model'].str.split('_', expand=True)[1].str.replace('seed=', '').astype(int)

all_res_agg

56795
55408


/tmp/ipykernel_1239380/2677695248.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_res_agg[['exp_name', 'base_model', 'exp_hp']] = split_values
/tmp/ipykernel_1239380/2677695248.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_res_agg[['exp_name', 'base_model', 'exp_hp']] = split_values
/tmp/ipykernel_1239380/2677695248.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

,Mode,Epoch,Train_Loss,Val_Loss,Val_NDCG,dataset,file_name,head,agg_method,exp_name,base_model,exp_hp,head_method,freeze,seed
0,external,0,2.145163e+31,2.367876e+32,0.070497,movielens_1m,item_not_error|model=sasrec_seed=0_final=items...,replace,exp_learnable_hawkes,item_not_error,model=sasrec_seed=0_final=items,agg=exp_learnable_hawkes__head=adding__freeze=...,adding,True,0
1,external,1,2.220826e+31,2.367876e+32,0.070497,movielens_1m,item_not_error|model=sasrec_seed=0_final=items...,replace,exp_learnable_hawkes,item_not_error,model=sasrec_seed=0_final=items,agg=exp_learnable_hawkes__head=adding__freeze=...,adding,True,0
2,external,2,2.262798e+31,2.367876e+32,0.070497,movielens_1m,item_not_error|model=sasrec_seed=0_final=items...,replace,exp_learnable_hawkes,item_not_error,model=sasrec_seed=0_final=items,agg=exp_learnable_hawkes__head=adding__freeze=...,adding,True,0
3,external,3,2.101723e+31,2.367876e+32,0.070497,movielens_1m,item_not_error|model=sasrec_seed=0_final=items...,replace,exp_learnable_hawkes,item_not_error,model=sasrec_seed=0_final=items,agg=exp_learnable_hawkes__head=adding__freeze=...,adding,True,0
4,external,4,2.416441e+31,2.367876e+32,0.070497,movielens_1m,item_not_error|model=sasrec_seed=0_final=items...,replace,exp_learnable_hawkes,item_not_error,model=sasrec_seed=0_final=items,agg=exp_learnable_hawkes__head=adding__freeze=...,adding,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,pretrain,145,5.692400e+00,6.324800e+00,0.071042,movielens_1m,item_not_error|model=sasrec_seed=2_final=items...,replace,no_agg,item_search,model=sasrec_seed=2_final=items,,no_head,False,2
146,pretrain,146,5.689800e+00,6.314500e+00,0.074196,movielens_1m,item_not_error|model=sasrec_seed=2_final=items...,replace,no_agg,item_search,model=sasrec_seed=2_final=items,,no_head,False,2
147,pretrain,147,5.685200e+00,6.294300e+00,0.074538,movielens_1m,item_not_error|model=sasrec_seed=2_final=items...,replace,no_agg,item_search,model=sasrec_seed=2_final=items,,no_head,False,2
148,pretrain,148,5.682900e+00,6.320600e+00,0.070382,movielens_1m,item_not_error|model=sasrec_seed=2_final=items...,replace,no_agg,item_search,model=sasrec_seed=2_final=items,,no_head,False,2


In [15]:
agg_gr_metrics = all_res_agg.groupby(['freeze', "head_method", "agg_method", "seed"], as_index=False) \
    .agg({'Val_NDCG': 'max'}) \
    .sort_values('Val_NDCG') \
    # .groupby(['freeze', "head_method", "agg_method"], as_index=False) \
    # .agg({'Val_NDCG': ['mean', 'std']}) \
    # .sort_values('Val_NDCG')
agg_gr_metrics['freeze'] = agg_gr_metrics['freeze'] == 'True'

agg_max_gr_metrics = agg_gr_metrics.groupby(['freeze', "head_method", "agg_method"], as_index=False).agg({'Val_NDCG': ['mean', 'std']})

agg_max_gr_metrics

freeze head_method             agg_method  Val_NDCG          
                                                  mean       std
0   False      adding              attention  0.079880  0.002244
1   False      adding             exp_hawkes  0.025057  0.002914
2   False      adding   exp_learnable_hawkes  0.068190  0.003132
3   False      adding       kernel_attention  0.080184  0.003128
4   False      adding    learnable_attention  0.080257  0.001786
5   False      adding                    max  0.080186  0.002580
6   False      adding                   mean  0.080607  0.001639
7   False      adding  symmetrical_attention  0.079761  0.002527
8   False     no_head                 no_agg  0.077519  0.002081
9   False        over              attention  0.080613  0.002627
10  False        over             exp_hawkes  0.011823  0.001064
11  False        over   exp_learnable_hawkes  0.051637  0.002360
12  False        over       kernel_attention  0.079698  0.002641
13  False        over    learnable_attention  0.078820  0.002551
14  False        over                    max  0.079589  0.001379
15  False        over                   mean  0.081252  0.002183
16  False        over  symmetrical_attention  0.078638  0.002493
17   True      adding              attention  0.077609  0.002376
18   True      adding             exp_hawkes  0.071278  0.002396
19   True      adding   exp_learnable_hawkes  0.068522  0.002261
20   True      adding       kernel_attention  0.078512  0.002234
21   True      adding    learnable_attention  0.078119  0.002148
22   True      adding                    max  0.077525  0.002024
23   True      adding                   mean  0.077565  0.002498
24   True      adding  symmetrical_attention  0.078336  0.001760
25   True        over              attention  0.049856  0.000985
26   True        over             exp_hawkes  0.052852  0.004532
27   True        over   exp_learnable_hawkes  0.050724  0.002753
28   True        over       kernel_attention  0.051500  0.001423
29   True        over    learnable_attention  0.050003  0.001696
30   True        over                    max  0.049532  0.000778
31   True        over                   mean  0.053942  0.000669
32   True        over  symmetrical_attention  0.050339  0.000805

In [16]:
agg_gr_metrics_fr = agg_max_gr_metrics[(agg_max_gr_metrics['freeze']) | (agg_max_gr_metrics['agg_method'] == 'no_agg')]
agg_gr_metrics_not_fr = agg_max_gr_metrics[~agg_max_gr_metrics['freeze']]


agg_gr_metrics_not_fr.sort_values([('Val_NDCG', 'mean')])

freeze head_method             agg_method  Val_NDCG          
                                                  mean       std
10  False        over             exp_hawkes  0.011823  0.001064
1   False      adding             exp_hawkes  0.025057  0.002914
11  False        over   exp_learnable_hawkes  0.051637  0.002360
2   False      adding   exp_learnable_hawkes  0.068190  0.003132
8   False     no_head                 no_agg  0.077519  0.002081
16  False        over  symmetrical_attention  0.078638  0.002493
13  False        over    learnable_attention  0.078820  0.002551
14  False        over                    max  0.079589  0.001379
12  False        over       kernel_attention  0.079698  0.002641
7   False      adding  symmetrical_attention  0.079761  0.002527
0   False      adding              attention  0.079880  0.002244
3   False      adding       kernel_attention  0.080184  0.003128
5   False      adding                    max  0.080186  0.002580
4   False      adding    learnable_attention  0.080257  0.001786
6   False      adding                   mean  0.080607  0.001639
9   False        over              attention  0.080613  0.002627
15  False        over                   mean  0.081252  0.002183

In [17]:
agg_gr_metrics_fr.sort_values([('Val_NDCG', 'mean')])

freeze head_method             agg_method  Val_NDCG          
                                                  mean       std
30   True        over                    max  0.049532  0.000778
25   True        over              attention  0.049856  0.000985
29   True        over    learnable_attention  0.050003  0.001696
32   True        over  symmetrical_attention  0.050339  0.000805
27   True        over   exp_learnable_hawkes  0.050724  0.002753
28   True        over       kernel_attention  0.051500  0.001423
26   True        over             exp_hawkes  0.052852  0.004532
31   True        over                   mean  0.053942  0.000669
19   True      adding   exp_learnable_hawkes  0.068522  0.002261
18   True      adding             exp_hawkes  0.071278  0.002396
8   False     no_head                 no_agg  0.077519  0.002081
22   True      adding                    max  0.077525  0.002024
23   True      adding                   mean  0.077565  0.002498
17   True      adding              attention  0.077609  0.002376
21   True      adding    learnable_attention  0.078119  0.002148
24   True      adding  symmetrical_attention  0.078336  0.001760
20   True      adding       kernel_attention  0.078512  0.002234

# item add alpha

In [3]:
path = 'checkpoints/movielens_1m'
all_files = [file for file in os.listdir(path)  if ".csv" in file ]
all_files

['item_add_alpha|model=sasrec_seed=0_final=items|agg=attention__head=over__freeze=True.csv',
 'item_add_alpha|model=sasrec_seed=2_final=items|agg=mean__head=over__freeze=True.csv',
 'item_add_alpha|model=sasrec_seed=1_final=items|agg=learnable_attention__head=over__freeze=True.csv',
 'item_add_alpha|model=sasrec_seed=0_final=items|agg=mean__head=adding__freeze=True.csv',
 'item_add_alpha|model=sasrec_seed=1_final=items|agg=symmetrical_attention__head=adding__freeze=False.csv',
 'item_add_alpha|model=sasrec_seed=1_final=items|agg=attention__head=over__freeze=False.csv',
 'item_add_alpha|model=sasrec_seed=2_final=items|agg=mean__head=adding__freeze=False.csv',
 'item_add_alpha|model=sasrec_seed=2_final=items|agg=learnable_attention__head=adding__freeze=True.csv',
 'item_add_alpha|model=sasrec_seed=0_final=items|agg=max__head=over__freeze=False.csv',
 'item_add_alpha|model=sasrec_seed=2_final=items|agg=symmetrical_attention__head=over__freeze=True.csv',
 'item_add_alpha|model=sasrec_seed=

In [43]:
all_res = pd.DataFrame([])

for file in all_files:
    df = pd.read_csv(os.path.join(path, file))
    df['dataset'] = 'movielens_1m'
    df['file_name'] = file
    if len(file.split('|')) > 1:
        hp = file.split('|')[1]
        method = hp
        method = method.replace('.csv', '').replace('agg=', '')
        df['head'] = 'replace'
        df['agg_method'] = method
    else:
        df['head'] = '-'
        df['agg_method'] = 'no_agg'

    all_res = pd.concat([all_res, df])

all_res_agg = all_res

exp_mask = all_res_agg['file_name'].str.split('|').apply(lambda x: len(x)) == 3
print(len(all_res_agg))
all_res_agg = all_res_agg[exp_mask]
print(len(all_res_agg))


split_values = all_res_agg['file_name'].str.split('|', n=2, expand=True)
all_res_agg[['exp_name', 'base_model', 'exp_hp']] = split_values


all_res_agg = all_res_agg[all_res_agg['exp_name'] == 'item_add_alpha']

all_res_agg['head_method'] =  all_res_agg['exp_hp'].str.split('__', n=2, expand=True)[1] \
    .str.split('__', expand=True)[0] \
    .str.replace('head=', '')

all_res_agg['agg_method'] =  all_res_agg['exp_hp'].str.split('__', n=2, expand=True)[0] \
    .str.split('__', expand=True)[0] \
    .str.replace('agg=', '')

all_res_agg['freeze'] =  all_res_agg['exp_hp'].str.split('__', n=2, expand=True)[2] \
    .str.split('__', expand=True)[0] \
    .str.replace('freeze=', '') \
    .str.replace('.csv', '') \


baselines = [f'item_search|model=sasrec_seed={seed}_final=items.csv' for seed in [0, 1, 2]]
all_res_baseline = all_res[all_res['file_name'].isin(baselines)]
all_res_baseline[['exp_name', 'exp_hp', 'head_method', 'agg_method', 'freeze']] = \
'item_search', '', 'no_head', 'no_agg', 'False'

all_res_baseline['base_model'] = all_res_baseline['file_name'].str.split('|', expand=True)[1].str.replace('.csv', '')

all_res_agg = pd.concat([all_res_agg, all_res_baseline])

all_res_agg['seed'] = all_res_agg['base_model'].str.split('_', expand=True)[1].str.replace('seed=', '').astype(int)

all_res_agg

52226
51289


/tmp/ipykernel_757129/3165385673.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_res_agg[['exp_name', 'base_model', 'exp_hp']] = split_values
/tmp/ipykernel_757129/3165385673.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_res_agg[['exp_name', 'base_model', 'exp_hp']] = split_values
/tmp/ipykernel_757129/3165385673.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

,Mode,Epoch,Train_Loss,Val_Loss,Val_NDCG,dataset,file_name,head,agg_method,exp_name,base_model,exp_hp,head_method,freeze,seed
0,external,0,51.5717,27.5399,0.036435,movielens_1m,item_add_alpha|model=sasrec_seed=0_final=items...,replace,attention,item_add_alpha,model=sasrec_seed=0_final=items,agg=attention__head=over__freeze=True.csv,over,True,0
1,external,1,20.3047,23.9404,0.039876,movielens_1m,item_add_alpha|model=sasrec_seed=0_final=items...,replace,attention,item_add_alpha,model=sasrec_seed=0_final=items,agg=attention__head=over__freeze=True.csv,over,True,0
2,external,2,21.2834,27.3697,0.030574,movielens_1m,item_add_alpha|model=sasrec_seed=0_final=items...,replace,attention,item_add_alpha,model=sasrec_seed=0_final=items,agg=attention__head=over__freeze=True.csv,over,True,0
3,external,3,23.4839,31.9119,0.028993,movielens_1m,item_add_alpha|model=sasrec_seed=0_final=items...,replace,attention,item_add_alpha,model=sasrec_seed=0_final=items,agg=attention__head=over__freeze=True.csv,over,True,0
4,external,4,25.5437,28.4716,0.033494,movielens_1m,item_add_alpha|model=sasrec_seed=0_final=items...,replace,attention,item_add_alpha,model=sasrec_seed=0_final=items,agg=attention__head=over__freeze=True.csv,over,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,pretrain,295,5.7898,6.4030,0.070902,movielens_1m,item_search|model=sasrec_seed=0_final=items.csv,replace,no_agg,item_search,model=sasrec_seed=0_final=items,,no_head,False,0
296,pretrain,296,5.7898,6.4302,0.068655,movielens_1m,item_search|model=sasrec_seed=0_final=items.csv,replace,no_agg,item_search,model=sasrec_seed=0_final=items,,no_head,False,0
297,pretrain,297,5.7945,6.4271,0.068272,movielens_1m,item_search|model=sasrec_seed=0_final=items.csv,replace,no_agg,item_search,model=sasrec_seed=0_final=items,,no_head,False,0
298,pretrain,298,5.7958,6.3907,0.068932,movielens_1m,item_search|model=sasrec_seed=0_final=items.csv,replace,no_agg,item_search,model=sasrec_seed=0_final=items,,no_head,False,0


In [44]:
agg_gr_metrics = all_res_agg.groupby(['freeze', "head_method", "agg_method", "seed"], as_index=False) \
    .agg({'Val_NDCG': 'max'}) \
    .sort_values('Val_NDCG') \
    # .groupby(['freeze', "head_method", "agg_method"], as_index=False) \
    # .agg({'Val_NDCG': ['mean', 'std']}) \
    # .sort_values('Val_NDCG')
agg_gr_metrics['freeze'] = agg_gr_metrics['freeze'] == 'True'

agg_gr_metrics = agg_gr_metrics.groupby(['freeze', "head_method", "agg_method"], as_index=False).agg({'Val_NDCG': ['mean', 'std']})

agg_gr_metrics

freeze head_method             agg_method  Val_NDCG          
                                                  mean       std
0   False      adding              attention  0.072975  0.002677
1   False      adding       kernel_attention  0.074117  0.005835
2   False      adding    learnable_attention  0.072584  0.002812
3   False      adding                    max  0.072953  0.002449
4   False      adding                   mean  0.073348  0.001449
5   False      adding  symmetrical_attention  0.073476  0.002275
6   False     no_head                 no_agg  0.073556  0.002489
7   False        over              attention  0.059695  0.007198
8   False        over       kernel_attention  0.066402  0.006144
9   False        over    learnable_attention  0.064084  0.005900
10  False        over                    max  0.066978  0.005416
11  False        over                   mean  0.059773  0.007063
12  False        over  symmetrical_attention  0.063019  0.004138
13   True      adding              attention  0.073737  0.002936
14   True      adding             exp_hawkes  0.067653  0.003426
15   True      adding       kernel_attention  0.074168  0.002938
16   True      adding    learnable_attention  0.073985  0.002907
17   True      adding                    max  0.073522  0.002465
18   True      adding                   mean  0.073784  0.002893
19   True      adding  symmetrical_attention  0.074118  0.003176
20   True        over              attention  0.057774  0.010525
21   True        over       kernel_attention  0.057055  0.011771
22   True        over    learnable_attention  0.058026  0.010089
23   True        over                    max  0.059315  0.007857
24   True        over                   mean  0.058147  0.009879
25   True        over  symmetrical_attention  0.056662  0.012452

In [45]:
agg_gr_metrics_fr = agg_gr_metrics[(agg_gr_metrics['freeze']) | (agg_gr_metrics['agg_method'] == 'no_agg')]
agg_gr_metrics_not_fr = agg_gr_metrics[~agg_gr_metrics['freeze']]


agg_gr_metrics_not_fr.sort_values([('Val_NDCG', 'mean')])

freeze head_method             agg_method  Val_NDCG          
                                                  mean       std
7   False        over              attention  0.059695  0.007198
11  False        over                   mean  0.059773  0.007063
12  False        over  symmetrical_attention  0.063019  0.004138
9   False        over    learnable_attention  0.064084  0.005900
8   False        over       kernel_attention  0.066402  0.006144
10  False        over                    max  0.066978  0.005416
2   False      adding    learnable_attention  0.072584  0.002812
3   False      adding                    max  0.072953  0.002449
0   False      adding              attention  0.072975  0.002677
4   False      adding                   mean  0.073348  0.001449
5   False      adding  symmetrical_attention  0.073476  0.002275
6   False     no_head                 no_agg  0.073556  0.002489
1   False      adding       kernel_attention  0.074117  0.005835

In [46]:
agg_gr_metrics_fr.sort_values([('Val_NDCG', 'mean')])

freeze head_method             agg_method  Val_NDCG          
                                                  mean       std
25   True        over  symmetrical_attention  0.056662  0.012452
21   True        over       kernel_attention  0.057055  0.011771
20   True        over              attention  0.057774  0.010525
22   True        over    learnable_attention  0.058026  0.010089
24   True        over                   mean  0.058147  0.009879
23   True        over                    max  0.059315  0.007857
14   True      adding             exp_hawkes  0.067653  0.003426
17   True      adding                    max  0.073522  0.002465
6   False     no_head                 no_agg  0.073556  0.002489
13   True      adding              attention  0.073737  0.002936
18   True      adding                   mean  0.073784  0.002893
16   True      adding    learnable_attention  0.073985  0.002907
19   True      adding  symmetrical_attention  0.074118  0.003176
15   True      adding       kernel_attention  0.074168  0.002938

In [21]:
agg_gr_metrics[(agg_gr_metrics['head_method'] == "adding") & (agg_gr_metrics['freeze'] == "False")]['agg_method'].unique()

array(['learnable_attention', 'kernel_attention', 'attention', 'max',
       'mean', 'symmetrical_attention'], dtype=object)

In [19]:
pd.crosstab(agg_gr_metrics['freeze'], agg_gr_metrics['head_method'])

head_method,adding,no_head,over
freeze,,,
False,0,1,0
False,6,0,6
True,7,0,6


In [3]:
path = 'checkpoints/movielens_1m'
all_files = [file for file in os.listdir(path)  if ".csv" in file ]
all_files

['model_sasrec_0|agg=exp_learnable_hawkes.csv',
 'model_sasrec_0.csv',
 'model_sasrec_0|agg=kernel_attention.csv',
 'model_sasrec_0|agg=max.csv',
 'model_sasrec_0|agg=attention.csv',
 'model_sasrec_0|agg=mean.csv',
 'model_sasrec_0|agg=learnable_attention.csv',
 'model_sasrec_0|agg=exp_hawkes.csv',
 'model_sasrec_0|agg=symmetrical_attention.csv']

In [9]:
all_res = pd.DataFrame([])

for file in all_files:
    df = pd.read_csv(os.path.join(path, file))
    df['dataset'] = 'movielens_1m'
    if len(file.split('|')) > 1:
        hp = file.split('|')[1]
        method = hp
        method = method.replace('.csv', '').replace('agg=', '')
        df['head'] = 'replace'
        df['agg_method'] = method
    else:
        df['head'] = '-'
        df['agg_method'] = 'no_agg'

    all_res = pd.concat([all_res, df])
    

all_res 

,Mode,Epoch,Train_Loss,Val_Loss,Val_NDCG,dataset,head,agg_method
0,external,0,2.133274e+32,5.853745e+33,0.014367,movielens_1m,replace,exp_learnable_hawkes
1,external,1,2.138385e+32,5.520207e+33,0.021319,movielens_1m,replace,exp_learnable_hawkes
2,external,2,2.162771e+32,5.308473e+33,0.026512,movielens_1m,replace,exp_learnable_hawkes
3,external,3,2.170690e+32,5.287630e+33,0.028058,movielens_1m,replace,exp_learnable_hawkes
4,external,4,2.143634e+32,5.285985e+33,0.029965,movielens_1m,replace,exp_learnable_hawkes
...,...,...,...,...,...,...,...,...
45,external,45,2.443200e+00,6.753100e+00,0.062938,movielens_1m,replace,symmetrical_attention
46,external,46,2.445400e+00,6.755600e+00,0.063468,movielens_1m,replace,symmetrical_attention
47,external,47,2.445900e+00,6.751300e+00,0.061841,movielens_1m,replace,symmetrical_attention
48,external,48,2.444300e+00,6.775500e+00,0.061479,movielens_1m,replace,symmetrical_attention


In [11]:
all_res.groupby("agg_method", as_index=False).agg({'Val_NDCG': 'max'}).sort_values('Val_NDCG')

,agg_method,Val_NDCG
2,exp_learnable_hawkes,0.043606
1,exp_hawkes,0.046789
3,kernel_attention,0.066620
7,no_agg,0.067446
8,symmetrical_attention,0.067895
5,max,0.068515
6,mean,0.068523
4,learnable_attention,0.068705
0,attention,0.069147


# renaming

In [6]:
# renaming

path = 'checkpoints/movielens_1m'
all_files = [file for file in os.listdir(path)  if "sarsrec" in file ]
all_files

new_name_files = []
for file in all_files:
    new_file = file
    new_file = new_file.replace('model=sarsrec_seed=0', 'new_head|model=sasrec_seed=0')
    new_name_files.append(new_file)


for old_file, new_file in zip(all_files, new_name_files):
    os.rename(os.path.join(path, old_file), os.path.join(path, new_file))

In [7]:
folder_path = "checkpoints/movielens_1m"  # замените на ваш путь

for file in Path(folder_path).iterdir():
    if file.is_file() and file.name.startswith('test'):
        created = datetime.datetime.fromtimestamp(file.stat().st_ctime)

        print(f"{file.name} - создан: {created.strftime('%d.%m.%Y')}")

test|model=sasrec_seed=0_final=items|agg=learnable_attention__head=adding__freeze=True.csv - создан: 26.09.2025
test|model=sasrec_seed=0_final=items|agg=exp_learnable_hawkes__head=over.yaml - создан: 26.09.2025
test|model=sasrec_seed=0_final=items|agg=symmetrical_attention__head=adding__freeze=True.csv - создан: 26.09.2025
test|model=sasrec_seed=0_final=items|agg=kernel_attention__head=over__freeze=False.csv - создан: 26.09.2025
test|model=sasrec_seed=0_final=items|agg=attention_hawkes__head=over.yaml - создан: 26.09.2025
test|model=sasrec_seed=0_final=items|agg=symmetrical_attention__head=over__freeze=False.pth - создан: 26.09.2025
test|model=sasrec_seed=0_final=items|agg=exp_hawkes__head=adding.yaml - создан: 26.09.2025
test|model=sasrec_seed=0_final=items|agg=kernel_attention__head=over__freeze=False.pth - создан: 26.09.2025
test|model=sasrec_seed=0_final=items|agg=kernel_attention__head=over.yaml - создан: 26.09.2025
test|model=sasrec_seed=0_final=items|agg=learnable_attention__hea

In [12]:
year = datetime.datetime.now().year
cutoff_date = datetime.datetime(year, 8, 24)
if datetime.datetime.now() < cutoff_date:
    cutoff_date = datetime.datetime(year - 1, 8, 24)

folder = Path(folder_path)
count = 0

old_name = 'iter_add_alpha'
new_name = 'item_add_alpha'

for file in folder.iterdir():
    if (file.is_file() and 
        file.name.startswith(old_name) and 
        datetime.datetime.fromtimestamp(file.stat().st_ctime) >= cutoff_date):
        
        new_file_name = file.name
        new_file_name = new_file_name.replace(old_name, new_name)
        file.rename(file.parent / new_file_name)
        print(f"Переименован: {file.name}")
        print(f"Новое имя: {new_file_name}")
        count += 1

Переименован: iter_add_alpha|model=sasrec_seed=0_final=items|agg=symmetrical_attention__head=adding.yaml
Новое имя: item_add_alpha|model=sasrec_seed=0_final=items|agg=symmetrical_attention__head=adding.yaml
Переименован: iter_add_alpha|model=sasrec_seed=0_final=items|agg=kernel_attention__head=adding__freeze=True.csv
Новое имя: item_add_alpha|model=sasrec_seed=0_final=items|agg=kernel_attention__head=adding__freeze=True.csv
Переименован: iter_add_alpha|model=sasrec_seed=0_final=items|agg=learnable_attention__head=over__freeze=False.pth
Новое имя: item_add_alpha|model=sasrec_seed=0_final=items|agg=learnable_attention__head=over__freeze=False.pth
Переименован: iter_add_alpha|model=sasrec_seed=0_final=items|agg=kernel_attention__head=over__freeze=True.pth
Новое имя: item_add_alpha|model=sasrec_seed=0_final=items|agg=kernel_attention__head=over__freeze=True.pth
Переименован: iter_add_alpha|model=sasrec_seed=0_final=items|agg=symmetrical_attention__head=over__freeze=True.pth
Новое имя: item